In [1]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17270, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17270 (delta 0), reused 0 (delta 0), pack-reused 17265 (from 1)
Receiving objects: 100% (17270/17270), 16.12 MiB | 9.18 MiB/s, done.
Resolving deltas: 100% (11858/11858), done.


In [2]:
!pip install -U pip
!pip install -r yolov5/requirements.txt
!pip install Pillow
!pip install opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 139.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 143.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 M

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [14]:
import tarfile
import os

# Path to your dataset .tgz file
tgz_file_path = '/content/drive/MyDrive/Yolo/102flowers.tgz'

# Directory to extract images
extracted_images_path = '/content/extracted_images/'

# Ensure the output directory exists
os.makedirs(extracted_images_path, exist_ok=True)

# Extract .tgz file
with tarfile.open(tgz_file_path, 'r:gz') as tar:
    tar.extractall(path=extracted_images_path)

print("Images extracted successfully.")


Images extracted successfully.


In [15]:
import os
import shutil
from pathlib import Path
import scipy.io
import random

# Define paths
source_dir = '/content/extracted_images/jpg/'  # Change this to the correct dataset path
target_base_dir = '/content/classified_dataset/'  # Output directory for classified images
mat_file_path = '/content/drive/MyDrive/Yolo/imagelabels.mat'  # Path to the .mat file

# Ensure target base directory exists
os.makedirs(target_base_dir, exist_ok=True)

# Load .mat file
mat_data = scipy.io.loadmat(mat_file_path)
labels = mat_data['labels'][0]  # Adjust indexing based on mat structure

# Get all image files
image_files = sorted([f for f in os.listdir(source_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])

# Move images to class-specific folders
image_class_map = {}
for img_file, label in zip(image_files, labels):
    class_label = str(label)  # Convert to string for folder naming
    class_dir = os.path.join(target_base_dir, class_label)
    os.makedirs(class_dir, exist_ok=True)
    shutil.move(os.path.join(source_dir, img_file), os.path.join(class_dir, img_file))
    image_class_map[img_file] = class_label

print(f"Dataset organized into {len(os.listdir(target_base_dir))} class folders.")

# Define train, validation, and test directories
train_dir = '/content/dataset/train/'
val_dir = '/content/dataset/val/'
test_dir = '/content/dataset/test/'

# Create directories for train, val, and test, ensuring 102 class subdirectories
for base_dir in [train_dir, val_dir, test_dir]:
    os.makedirs(base_dir, exist_ok=True)
    for class_label in set(image_class_map.values()):
        os.makedirs(os.path.join(base_dir, class_label), exist_ok=True)

# Shuffle image files for random split
image_files = list(image_class_map.keys())
random.shuffle(image_files)

# Split ratios
train_ratio = 0.5
val_ratio = 0.25
test_ratio = 0.25

# Calculate split indices
num_images = len(image_files)
num_train = int(train_ratio * num_images)
num_val = int(val_ratio * num_images)

# Split the filenames
train_filenames = image_files[:num_train]
val_filenames = image_files[num_train:num_train + num_val]
test_filenames = image_files[num_train + num_val:]

# Function to move images to respective directories
def move_images(filenames, src_base_dir, dst_base_dir):
    for filename in filenames:
        class_label = image_class_map[filename]
        src_path = os.path.join(src_base_dir, class_label, filename)
        dst_path = os.path.join(dst_base_dir, class_label, filename)
        shutil.move(src_path, dst_path)
        print(f"Moved {filename} to {dst_path}")

# Move images to train, val, and test directories
move_images(train_filenames, target_base_dir, train_dir)
move_images(val_filenames, target_base_dir, val_dir)
move_images(test_filenames, target_base_dir, test_dir)

print("Dataset split into train, validation, and test sets successfully.")


Streaming output truncated to the last 5000 lines.
Moved image_02969.jpg to /content/dataset/train/60/image_02969.jpg
Moved image_06429.jpg to /content/dataset/train/9/image_06429.jpg
Moved image_00133.jpg to /content/dataset/train/77/image_00133.jpg
Moved image_00923.jpg to /content/dataset/train/81/image_00923.jpg
Moved image_05940.jpg to /content/dataset/train/68/image_05940.jpg
Moved image_03337.jpg to /content/dataset/train/8/image_03337.jpg
Moved image_04045.jpg to /content/dataset/train/12/image_04045.jpg
Moved image_06082.jpg to /content/dataset/train/14/image_06082.jpg
Moved image_02282.jpg to /content/dataset/train/41/image_02282.jpg
Moved image_03027.jpg to /content/dataset/train/60/image_03027.jpg
Moved image_06293.jpg to /content/dataset/train/61/image_06293.jpg
Moved image_05301.jpg to /content/dataset/train/70/image_05301.jpg
Moved image_03359.jpg to /content/dataset/train/8/image_03359.jpg
Moved image_02295.jpg to /content/dataset/train/41/image_02295.jpg
Moved image_02

In [6]:
%cd yolov5


/content/yolov5


In [17]:
!python classify/train.py --data /content/dataset --epochs 10 --img 224 --name flowers1

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-01-31 11:25:18.172734: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738322718.193277    8463 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738322718.199573    8463 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
classify/train: model=yolov5s-cls.pt, data=/content/dataset, epochs=10, batch_

In [18]:
!python classify/val.py --weights runs/train-cls/flowers1/weights/best.pt --data /content/dataset/ --img 224


classify/val: data=/content/dataset/, weights=['runs/train-cls/flowers1/weights/best.pt'], batch_size=128, imgsz=224, device=, workers=8, verbose=True, project=runs/val-cls, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-398-g5cdad892 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 117 layers, 4297350 parameters, 0 gradients, 10.5 GFLOPs
testing:   0% 0/16 [00:00<?, ?it/s]/content/yolov5/classify/val.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type != "cpu"):
testing: 100% 16/16 [00:05<00:00,  2.75it/s]
                   Class      Images    top1_acc    top5_acc
                     all        2048       0.717        0.91
                       1          13       0.846           1
                      10          13       0.538           1
                     100          12       0.583    